In [2]:
import re
import pandas as pd

In [143]:
# humploc_predictions_txt = "humploc_predictions.txt"
# in uniprot file, "subcellular location" can be empty

linecounter = 0
secondcounter = 0 
counter = 0

h_ac = None
h_id = None
h_seq = "" 
h_loc = None 
h_scores = None

subcellular_location_tsv = "subcellular_location.tsv"
humploc_predictions_txt = "humploc_predictions.tsv" # tsv better than txt (because I removed the header info manually; this should be in the code instead) Also, tsv has a correction (two newlines removed into one)
uniprot_results_tsv = "uniprot_results.tsv"
output_filename = "comparison_output.csv"

begin_flag = True
seq_flag = False
dic = {}

df = pd.DataFrame(columns=['AC', 'ID', 'SEQ', 'PRED_LOC', 'SCORES'])

with open(humploc_predictions_txt) as humploc_text:
    for line in humploc_text: 
        counter += 1
        linecounter += 1
        if not begin_flag:
            secondcounter += 1
        if linecounter == 1:
            h_ac = line.strip()
        if linecounter == 2:
            h_id = line.strip()
        if linecounter == 3:
            seq_flag = True
        if not line.strip():
            seq_flag = False
            begin_flag = False 
        if seq_flag:
            h_seq = h_seq + line.strip()
        if secondcounter == 1:
            h_loc = line.strip()
        if secondcounter == 2: 
            h_scores = line.strip()
        if re.search('//', line):
            linecounter = 0
            secondcounter = 0
            begin_flag = True
            dic['AC'] = h_ac
            dic['ID'] = h_id
            dic['SEQ'] = h_seq
            dic['PRED_LOC'] = h_loc
            dic['SCORES'] = h_scores
            h_ac = None
            h_id = None
            h_seq = "" 
            h_loc = None 
            h_scores = None
            df = df.append(dic, ignore_index=True)
            dic = {}
print('created a Dataframe from Humploc data')

created a Dataframe from Humploc data


In [144]:
# todo: 
# - the following needs to be implemented: we want to exclude certain rows from the Humploc Dataframe, based on prediction quality
# - also, for humploc: validated, supported and approved are experimental (DeepLoc presentation points to this), need to filter that, too
def check_score_of_humploc_prediction(x): #  todo: define a cutoff, based on the 12 prediction scores from humploc
    return True

In [145]:
humploc_df = df.fillna('') # subtitute NaNs with ''
humploc_df

,AC,ID,SEQ,PRED_LOC,SCORES
0,AC,ID,SEQ,Predicted Subcellular location by Hum-mPLoc 3.0,Outputted prediction scores for each location ...
1,P62258,1433E_HUMAN,MDDREDLVYQAKLAEQAERYDEMVESMKKVAGMDVELTVEERNLLS...,Cytoplasm.,"-1.158095,1.053253,-1.496686,-2.250498,-1.8343..."
2,Q04917,1433F_HUMAN,MGDREQLLQRARLAEQAERYDDMASAMKAVTELNEPLSNEDRNLLS...,Cytoplasm.,"-1.254769,0.487241,-1.091541,-2.543192,-2.0037..."
3,P61981,1433G_HUMAN,MVDREQLVQKARLAEQAERYDDMAAAMKNVTELNEPLSNEERNLLS...,Cytoplasm.,"-1.222702,0.689057,-1.099006,-2.577572,-2.0572..."
4,P31947,1433S_HUMAN,MERASLIQKAKLAEQAERYEDMAAFMKGAVEKGEELSCEERNLLSV...,Cytoplasm.,"-1.241884,1.438412,-1.192020,-2.279416,-1.7292..."
5,P27348,1433T_HUMAN,MEKTELIQKAKLAEQAERYDDMATCMKAVTEQGAELSNEERNLLSV...,Cytoplasm.,"-1.173382,2.196761,-1.051799,-1.719705,-1.7929..."
6,P63104,1433Z_HUMAN,MDKNELVQKAKLAEQAERYDDMAACMKSVTEQGAELSNEERNLLSV...,Cytoplasm.,"-1.169426,2.205039,-1.041187,-1.750936,-1.8200..."
7,P30443,1A01_HUMAN,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,Endosome. Golgi apparatus.,"-1.825552,-4.784307,-2.787740,0.483626,-0.0560..."
8,P01892,1A02_HUMAN,MAVMAPRTLVLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,Endosome. Endoplasmic reticulum. Golgi appar...,"-1.778965,-4.829878,-2.702989,0.544520,0.11073..."
9,P04439,1A03_HUMAN,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,Endosome. Endoplasmic reticulum. Golgi appar...,"-1.845178,-4.747686,-2.862900,0.513883,0.01748..."


In [146]:
hpa = pd.read_csv(subcellular_location_tsv, sep="\t")
hpa = hpa.fillna('')

In [147]:
uniprot = pd.read_csv(uniprot_results_tsv, sep="\t")
uniprot = uniprot.fillna('')

In [148]:
# inner join on the AC identifier using humploc and uniprot results
proteins_in_uniprot_and_humploc_using_AC = pd.merge(uniprot, humploc_df, left_on="Entry", right_on="AC", how="inner")

In [150]:
df = proteins_in_uniprot_and_humploc_using_AC
df

,Entry,Entry name,Status,Protein names,Gene names,Subcellular location [CC],Gene ontology (GO),AC,ID,SEQ,PRED_LOC,SCORES
0,P04637,P53_HUMAN,reviewed,Cellular tumor antigen p53 (Antigen NY-CO-13) ...,TP53 P53,SUBCELLULAR LOCATION: Cytoplasm. Nucleus. Nucl...,cytoplasm [GO:0005737]; cytosol [GO:0005829]; ...,P04637,P53_HUMAN,MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLS...,Nucleus.,"-1.421297,-0.829262,-1.701711,-2.569025,-1.864..."
1,Q14524,SCN5A_HUMAN,reviewed,Sodium channel protein type 5 subunit alpha (H...,SCN5A,SUBCELLULAR LOCATION: Cell membrane {ECO:00002...,caveola [GO:0005901]; cell surface [GO:0009986...,Q14524,SCN5A_HUMAN,MANFLLPRGTSSFRRFTRESLAAIEKRMAEKQARGSTTLQESREGL...,Plasma membrane.,"-1.826585,-1.933985,-1.308556,-2.743710,-2.408..."
2,P05067,A4_HUMAN,reviewed,Amyloid-beta A4 protein (ABPP) (APPI) (APP) (A...,APP A4 AD1,SUBCELLULAR LOCATION: Membrane; Single-pass ty...,amyloid-beta complex [GO:0106003]; apical part...,P05067,A4_HUMAN,MLPGLALLLLAAWTARALEVPTDGNAGLLAEPQIAMFCGRLNMHMN...,Extracellular.,"-1.675579,-1.089203,-1.628664,-1.264180,-0.822..."
3,P00533,EGFR_HUMAN,reviewed,Epidermal growth factor receptor (EC 2.7.10.1)...,EGFR ERBB ERBB1 HER1,SUBCELLULAR LOCATION: Cell membrane; Single-pa...,apical plasma membrane [GO:0016324]; basolater...,P00533,EGFR_HUMAN,MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...,Plasma membrane.,"-1.241858,-2.066043,-1.782397,-0.825578,-1.115..."
4,P35555,FBN1_HUMAN,reviewed,Fibrillin-1 [Cleaved into: Asprosin],FBN1 FBN,SUBCELLULAR LOCATION: Secreted {ECO:0000269|Pu...,basement membrane [GO:0005604]; endoplasmic re...,P35555,FBN1_HUMAN,MRRGRLLEIALGFTVLLASYTSHGADANLEAGNVKETRASRAKRRG...,Plasma membrane.,"-1.593076,-1.352582,-2.480642,-2.095503,-2.023..."
5,P35222,CTNB1_HUMAN,reviewed,Catenin beta-1 (Beta-catenin),CTNNB1 CTNNB OK/SW-cl.35 PRO2286,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000269|P...,adherens junction [GO:0005912]; apical part of...,P35222,CTNB1_HUMAN,MATQADLMELDMAMEPDRKAAVSHWQQQSYLDSGIHSGATTTAPSL...,Cytoplasm.,"-0.909892,2.289614,-0.170275,-2.687533,-2.5651..."
6,P10275,ANDR_HUMAN,reviewed,Androgen receptor (Dihydrotestosterone recepto...,AR DHTR NR3C4,SUBCELLULAR LOCATION: Nucleus {ECO:0000269|Pub...,cytoplasm [GO:0005737]; cytosol [GO:0005829]; ...,P10275,ANDR_HUMAN,MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAAS...,Nucleus.,"-1.609735,-0.518090,-2.641733,-2.268793,-1.834..."
7,P00451,FA8_HUMAN,reviewed,Coagulation factor VIII (Antihemophilic factor...,F8 F8C,"SUBCELLULAR LOCATION: Secreted, extracellular ...",COPII-coated ER to Golgi transport vesicle [GO...,P00451,FA8_HUMAN,MQIELSTCFFLCLLRFCFSATRRYYLGAVELSWDYMQSDLGELPVD...,Endoplasmic reticulum. Golgi apparatus.,"-1.286432,-2.423427,-1.796176,-1.866689,0.5352..."
8,Q5S007,LRRK2_HUMAN,reviewed,Leucine-rich repeat serine/threonine-protein k...,LRRK2 PARK8,SUBCELLULAR LOCATION: Membrane; Peripheral mem...,amphisome [GO:0044753]; autolysosome [GO:00447...,Q5S007,LRRK2_HUMAN,MASGSCQGCEEDEETLKKLIVRLNNVQEGKQIETLVQILEDLLVFT...,Cytoplasm.,"-1.543625,1.100506,-1.454457,-1.534116,-1.0217..."
9,Q8WZ42,TITIN_HUMAN,reviewed,Titin (EC 2.7.11.1) (Connectin) (Rhabdomyosarc...,TTN,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000305|P...,condensed nuclear chromosome [GO:0000794]; cyt...,Q8WZ42,TITIN_HUMAN,MTTQAPTFTQPLQSVVVLEGSTATFEAHISGFPVPEVSWFRDGQVI...,Cytoplasm. Cytoskeleton. Plasma membrane.,"-1.418561,2.064323,0.189796,-1.398633,-2.24081..."


In [151]:
mask = pd.DataFrame({
           'UNIPROTS': (df["Subcellular location [CC]"] + df["Gene ontology (GO)"]).str.lower().map(set),
           'HUMPLOC': df["PRED_LOC"].str.lower().map(set)
       }).apply(lambda row: bool(row['UNIPROTS'].intersection(row['HUMPLOC'])), axis=1)

df[mask].reset_index(drop=True)

,Entry,Entry name,Status,Protein names,Gene names,Subcellular location [CC],Gene ontology (GO),AC,ID,SEQ,PRED_LOC,SCORES
0,P04637,P53_HUMAN,reviewed,Cellular tumor antigen p53 (Antigen NY-CO-13) ...,TP53 P53,SUBCELLULAR LOCATION: Cytoplasm. Nucleus. Nucl...,cytoplasm [GO:0005737]; cytosol [GO:0005829]; ...,P04637,P53_HUMAN,MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLS...,Nucleus.,"-1.421297,-0.829262,-1.701711,-2.569025,-1.864..."
1,Q14524,SCN5A_HUMAN,reviewed,Sodium channel protein type 5 subunit alpha (H...,SCN5A,SUBCELLULAR LOCATION: Cell membrane {ECO:00002...,caveola [GO:0005901]; cell surface [GO:0009986...,Q14524,SCN5A_HUMAN,MANFLLPRGTSSFRRFTRESLAAIEKRMAEKQARGSTTLQESREGL...,Plasma membrane.,"-1.826585,-1.933985,-1.308556,-2.743710,-2.408..."
2,P05067,A4_HUMAN,reviewed,Amyloid-beta A4 protein (ABPP) (APPI) (APP) (A...,APP A4 AD1,SUBCELLULAR LOCATION: Membrane; Single-pass ty...,amyloid-beta complex [GO:0106003]; apical part...,P05067,A4_HUMAN,MLPGLALLLLAAWTARALEVPTDGNAGLLAEPQIAMFCGRLNMHMN...,Extracellular.,"-1.675579,-1.089203,-1.628664,-1.264180,-0.822..."
3,P00533,EGFR_HUMAN,reviewed,Epidermal growth factor receptor (EC 2.7.10.1)...,EGFR ERBB ERBB1 HER1,SUBCELLULAR LOCATION: Cell membrane; Single-pa...,apical plasma membrane [GO:0016324]; basolater...,P00533,EGFR_HUMAN,MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...,Plasma membrane.,"-1.241858,-2.066043,-1.782397,-0.825578,-1.115..."
4,P35555,FBN1_HUMAN,reviewed,Fibrillin-1 [Cleaved into: Asprosin],FBN1 FBN,SUBCELLULAR LOCATION: Secreted {ECO:0000269|Pu...,basement membrane [GO:0005604]; endoplasmic re...,P35555,FBN1_HUMAN,MRRGRLLEIALGFTVLLASYTSHGADANLEAGNVKETRASRAKRRG...,Plasma membrane.,"-1.593076,-1.352582,-2.480642,-2.095503,-2.023..."
5,P35222,CTNB1_HUMAN,reviewed,Catenin beta-1 (Beta-catenin),CTNNB1 CTNNB OK/SW-cl.35 PRO2286,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000269|P...,adherens junction [GO:0005912]; apical part of...,P35222,CTNB1_HUMAN,MATQADLMELDMAMEPDRKAAVSHWQQQSYLDSGIHSGATTTAPSL...,Cytoplasm.,"-0.909892,2.289614,-0.170275,-2.687533,-2.5651..."
6,P10275,ANDR_HUMAN,reviewed,Androgen receptor (Dihydrotestosterone recepto...,AR DHTR NR3C4,SUBCELLULAR LOCATION: Nucleus {ECO:0000269|Pub...,cytoplasm [GO:0005737]; cytosol [GO:0005829]; ...,P10275,ANDR_HUMAN,MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAAS...,Nucleus.,"-1.609735,-0.518090,-2.641733,-2.268793,-1.834..."
7,P00451,FA8_HUMAN,reviewed,Coagulation factor VIII (Antihemophilic factor...,F8 F8C,"SUBCELLULAR LOCATION: Secreted, extracellular ...",COPII-coated ER to Golgi transport vesicle [GO...,P00451,FA8_HUMAN,MQIELSTCFFLCLLRFCFSATRRYYLGAVELSWDYMQSDLGELPVD...,Endoplasmic reticulum. Golgi apparatus.,"-1.286432,-2.423427,-1.796176,-1.866689,0.5352..."
8,Q5S007,LRRK2_HUMAN,reviewed,Leucine-rich repeat serine/threonine-protein k...,LRRK2 PARK8,SUBCELLULAR LOCATION: Membrane; Peripheral mem...,amphisome [GO:0044753]; autolysosome [GO:00447...,Q5S007,LRRK2_HUMAN,MASGSCQGCEEDEETLKKLIVRLNNVQEGKQIETLVQILEDLLVFT...,Cytoplasm.,"-1.543625,1.100506,-1.454457,-1.534116,-1.0217..."
9,Q8WZ42,TITIN_HUMAN,reviewed,Titin (EC 2.7.11.1) (Connectin) (Rhabdomyosarc...,TTN,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000305|P...,condensed nuclear chromosome [GO:0000794]; cyt...,Q8WZ42,TITIN_HUMAN,MTTQAPTFTQPLQSVVVLEGSTATFEAHISGFPVPEVSWFRDGQVI...,Cytoplasm. Cytoskeleton. Plasma membrane.,"-1.418561,2.064323,0.189796,-1.398633,-2.24081..."


In [ ]:
# todo: we need to find a way to do an inner join on humploc and HPA, but for that, we need to somehow look up for HPA the corresponding Uniprot identifiers

In [5]:
# this block is probably unnecessary, have it here for reference

# def main(input_filename, output_filename):
#     with open(output_filename, 'w') as output_file:
#         write_stats(stats, output_file)
#         
# def analyze_humploc(f):
#     humploc_loc_re = re.compile(r'Centrosome\b | \bCytoplasm\b | \bCytoskeleton\b | \bEndoplasmic reticulum\b | \bEndosome\b | \bExtracellular\b | \bGolgi apparatus\b | \bLysosome\b | \bMitochondrion\b | \bNucleus\b | \bPeroxisome\b | \bPlasma membrane', flags=re.I | re.X)
#     line_number = 0
#     out_line_number = 1
#     humploc_id = None
#     humploc_ac = None
#     humploc_loc = None
#     for line in f:
#         if re.search("//", line):
#             line_number = 0
#         else:
#             line_number += 1
#         if line_number == 1:
#             humploc_ac = line
#         humploc_id = re.search('(.+)_HUMAN', line)
#         humploc_loc = humploc_loc_re.findall(line)
#         if humploc_id and humploc_loc and humploc_ac:
#             out_line_number += 1
#             
# # probably unnecessary block
# 
# cols = None
# unis = None
# hpas = None 
# 
# unis = list(uniprot.columns)
# unis = ["UNIPROT_" + s for s in unis]
# 
# hpas = list(hpa.columns)
# hpas = ["HPA_" +  s for s in hpas]
# 
# cols = [unis, hpas]
# 
# cols = [item for sublist in cols for item in sublist]
# cols = ['HUMPLOC_AC', 'HUMPLOC_ID', 'HUMPLOC_LOC'] + cols # this is probably wrong, and the whole block should be redone
# 
# out = pd.DataFrame(columns=cols)
# out